In [1]:
from config import project_config as config
from utils.sleep_wake_filter import filter_sleep_series
import pandas as pd
import numpy as np
from functools import reduce
from utils.data_utils import read_sleep_dairies_v2

merged_sources_path = 'Results/merged_indicators'
label = 'AWS Sleep'
models = ['pred_PSG-CNN', 'Biobank Sleep']
sleep_diaries_path = 'data/Sleep diaries'
diaries_df = read_sleep_dairies_v2(sleep_diaries_path, include_naps=False)

results = pd.DataFrame()
all_preds = pd.DataFrame()
for id in config['subject_ids']:

    subject_diary = diaries_df[diaries_df['subject_id'] == id]

    preds_df = pd.read_csv(f'{merged_sources_path}/sub_{id:02d}.csv')
    preds_df['epoch_ts'] = pd.to_datetime(preds_df['epoch_ts'])
    # df = df.dropna(subset=[label] + models)  # Drop epochs without a label or prediction
    preds_df.insert(0, 'subject_id', id)
    
    # df['pred_AWS-CNN'] = filter_sleep_series(df['pred_AWS-CNN'])
    # df['pred_PSG-CNN'] = filter_sleep_series(df['pred_PSG-CNN'])
    
    # Here we mark the epochs between sleep start and sleep end as recorded in sleep diary
    # This is how it's done:
    # - Create a column that's =1 for sleep_start epochs
    # - Create a column that's =-1 for sleep_end epochs
    # - Combine the two column so that the new "lights_off_period" column has a 1 when sleep start and a -1 when it ends
    # - Then find the cumulative sum of the lights_off_period column. The cumsum will be 1 between sleep start and sleep end
    #     and 0 elsewhere
    preds_df['lights_off_time'] = preds_df['epoch_ts'].isin(subject_diary['lights_off']).astype(int)
    preds_df['lights_on_time'] = preds_df['epoch_ts'].isin(subject_diary['lights_on']).astype(int).map({0: 0, 1: -1})  # Mark end of sleep with -1
    
    # merge the two columns. We can simply add them, because they are never non-zero on the same row. i.e. start timestamp and end timestamp are never the same
    preds_df['lights_off_period'] = preds_df['lights_off_time'] + preds_df['lights_on_time']
    preds_df['lights_off_period'] = preds_df['lights_off_period'].cumsum()

    # Next, create a column that assigns a distinct id to each sleep episode
    preds_df['sleep_episode_counter'] = preds_df['lights_off_time'].cumsum()  # This is a helper variable that creates a new id evey time sleep starts
    preds_df['sleep_episode_id'] = preds_df['sleep_episode_counter'].where(preds_df['lights_off_period'] == 1, 0)

    preds_df = preds_df.drop(columns=['lights_off_time', 'lights_on_time', 'sleep_episode_counter'])

    all_preds = pd.concat([all_preds, preds_df])


In [2]:
# Metrics computed over CV epochs only
from sklearn.metrics import classification_report
light_off_preds = all_preds[all_preds['lights_off_period'] == 1]
# temp_df = light_off_preds.dropna(subset=['PSG Sleep', 'Biobank Sleep'])
temp_df = light_off_preds.dropna(subset=['AWS Sleep', 'Biobank Sleep', 'pred_PSG-CNN'])
temp_df = temp_df[temp_df['is_cv_prediction'] == 1]

# print('AWS Model')
# print(classification_report(y_true=temp_df['AWS Sleep'], y_pred=temp_df['pred_AWS-CNN']))
# print('Biobank')
# print(classification_report(y_true=temp_df['AWS Sleep'], y_pred=temp_df['Biobank Sleep']))

In [3]:
# from sklearn.metrics import classification_report
# light_off_preds = all_preds[all_preds['lights_off_period'] == 1]
# # temp_df = light_off_preds.dropna(subset=['PSG Sleep', 'Biobank Sleep'])
# temp_df = light_off_preds.dropna(subset=['AWS Sleep', 'Biobank Sleep', 'pred_AWS-CNN'])

# print('AWS Model')
# print(classification_report(y_true=temp_df['AWS Sleep'], y_pred=temp_df['pred_AWS-CNN']))
# print('Biobank')
# print(classification_report(y_true=temp_df['AWS Sleep'], y_pred=temp_df['Biobank Sleep']))

# Sleep Summary Metrics

In [4]:
def calculate_sleep_summary_metrics(preds_df, source_col_name):
    
    # Sleep quality metrics are only calculated during a certain window
    # See cells above where this column was created
    lights_off_fltr = (preds_df['lights_off_period'] == 1)

    # # # # # # # # # # # # # # # # # # # # 
    # #   TRT: Total Recording Time
    # #   Defined as the time in minutes from lights out to
    # #   lights on. This time constitutes the sleep opportunity period.
    # #   Epochs of sleep are only scored between these two points.
    # # # # # # # # # # # # # # # # # # # # 

    TRT_df = preds_df[lights_off_fltr].groupby(['subject_id', 'sleep_episode_id'])['epoch_ts'].count() / 2
    TRT_df = TRT_df.reset_index().rename({'epoch_ts': 'TRT (Min)'}, axis=1)

    # # # # # # # # # # # # # # # # # # # # 
    # #   TST: Total Sleep Time
    # #   Defined as the time in minutes scored as NREM or REM
    # #   but excluding epochs of Unsure and Wake within the period between lights off and lights on
    # # # # # # # # # # # # # # # # # # # # 

    TST_df = preds_df[lights_off_fltr].groupby(['subject_id', 'sleep_episode_id'])[source_col_name].sum() / 2  # each epoch is half a minute
    TST_df = TST_df.reset_index().rename({source_col_name: 'TST (Min)'}, axis=1)

    # # # # # # # # # # # # # # # # # # # # 
    # #   SOL: Sleep Onset Latency
    # #   Defined as the time in minutes occurring from lights off to the first epoch of NREM or REM
    # # # # # # # # # # # # # # # # # # # # 

    lights_off_df = preds_df[lights_off_fltr].groupby(['subject_id', 'sleep_episode_id'])['epoch_ts'].min().\
        reset_index().rename({'epoch_ts': 'lights_off_time'}, axis=1)

    transition_col = f'{source_col_name} transition'
    preds_df[transition_col] = preds_df.groupby(['subject_id', 'sleep_episode_id'])[source_col_name].diff()

    falling_asleep_fltr = (preds_df[transition_col] == 1)
    # find the first epoch after lights_off_period, when we transition from wake to sleep
    sleep_onset_df = preds_df[lights_off_fltr & falling_asleep_fltr].groupby(['subject_id', 'sleep_episode_id'])['epoch_ts'].min().\
        reset_index().rename({'epoch_ts': 'sleep_onset'}, axis=1)

    SOL_df = pd.merge(
        left=lights_off_df,
        right=sleep_onset_df,
        on=['subject_id', 'sleep_episode_id'],
        how='left'
    )
    SOL_df['SOL (Min)'] = (SOL_df['sleep_onset'] - SOL_df['lights_off_time']).dt.seconds / 60  # convert to minutes
    SOL_df = SOL_df.drop(['lights_off_time'], axis=1)  # keep sleep_onset for WASO

    # # # # # # # # # # # # # # # # # # # # 
    # #   WASO: Wake After Sleep Onset
    # # # # # # # # # # # # # # # # # # # # 

    # We need to find first wake after sleep onset. Let's first remove epochs before sleep onset
    first_wake_df = pd.merge(
        left=preds_df[lights_off_fltr],
        right=SOL_df.drop('SOL (Min)', axis=1),  # don't need this column
        on=['subject_id', 'sleep_episode_id'],
        how='left'
    )
    # Sleep onset column is specific to each subject and sleep episode
    # So, we don't need to worry about grouping or filtering by these here
    # This is a row-wise comparison
    first_wake_df = first_wake_df[first_wake_df['epoch_ts'] > first_wake_df['sleep_onset']]  # remove epochs that occur before sleep onset
    first_wake_df = first_wake_df[first_wake_df[transition_col] == -1].groupby(['subject_id', 'sleep_episode_id'])['epoch_ts'].min().\
        reset_index().rename({'epoch_ts': 'first_wake'}, axis=1)

    # bring in sleep onset timestamp to calculate time between sleep onset and first wake
    WASO_df = pd.merge(
        left=first_wake_df,
        right=sleep_onset_df,
        on=['subject_id', 'sleep_episode_id'],
        how='left'
    )

    WASO_df['WASO (Min)'] = (WASO_df['first_wake'] - WASO_df['sleep_onset']).dt.seconds / 60  # convert to mintues
    WASO_df = WASO_df.drop(['first_wake', 'sleep_onset'], axis=1)

    # Now that we're done with WASO we can drop sleep_onset from SOL
    SOL_df = SOL_df.drop('sleep_onset', axis=1)

    # # # # # # # # # # # # # # # # # # # # 
    # #   SEEF: Sleep Efficiency
    # #   Defined as the percentage of TST against TRT
    # # # # # # # # # # # # # # # # # # # # 

    SEEF_df = pd.merge(
        left=TRT_df,
        right=TST_df,
        on=['subject_id', 'sleep_episode_id'],
        how='left'
    )

    SEEF_df['SEEF'] = SEEF_df['TST (Min)'] / SEEF_df['TRT (Min)']
    SEEF_df = SEEF_df.drop(['TRT (Min)', 'TST (Min)'], axis=1)
    
    metrics_list = [TRT_df, TST_df, SEEF_df, SOL_df, WASO_df]

    merge_fn = lambda l, r: pd.merge(
        left=l,
        right=r,
        on=['subject_id', 'sleep_episode_id'],
        how='left'
    )

    sleep_summary_metrics = reduce(merge_fn, metrics_list)

    return sleep_summary_metrics

In [5]:
def calculate_sleep_summary_metrics_v2(preds_df, source_col_name):
    
    # # # 
    # This v2 function calculates SOL and WASO in a different way
    # Instead of looking for a transition to sleep (wake for WASO)
    # It simply takes the first lights_off_period sleep epoch as sleep onset
    # And the first wake epoch after sleep onset as first-wake
    # This probably won't affect WASO, but will certainly change SOL
    # It changes SOL because in many cases the very first epoch of the
    # lights_off_period window is already a sleep epoch and there is no transition
    # inside the lights_off_period window

    # Sleep quality metrics are only calculated during a certain window
    # See cells above where this column was created
    lights_off_fltr = (preds_df['lights_off_period'] == 1)

    # # # # # # # # # # # # # # # # # # # # 
    # #   TRT: Total Recording Time
    # #   Defined as the time in minutes from lights out to
    # #   lights on. This time constitutes the sleep opportunity period.
    # #   Epochs of sleep are only scored between these two points.
    # # # # # # # # # # # # # # # # # # # # 

    TRT_df = preds_df[lights_off_fltr].groupby(['subject_id', 'sleep_episode_id'])['epoch_ts'].count() / 2
    TRT_df = TRT_df.reset_index().rename({'epoch_ts': 'TRT (Min)'}, axis=1)

    # # # # # # # # # # # # # # # # # # # # 
    # #   TST: Total Sleep Time
    # #   Defined as the time in minutes scored as NREM or REM
    # #   but excluding epochs of Unsure and Wake within the period between lights off and lights on
    # # # # # # # # # # # # # # # # # # # # 

    TST_df = preds_df[lights_off_fltr].groupby(['subject_id', 'sleep_episode_id'])[source_col_name].sum() / 2  # each epoch is half a minute
    TST_df = TST_df.reset_index().rename({source_col_name: 'TST (Min)'}, axis=1)

    # # # # # # # # # # # # # # # # # # # # 
    # #   SOL: Sleep Onset Latency
    # #   Defined as the time in minutes occurring from lights off to the first epoch of NREM or REM
    # # # # # # # # # # # # # # # # # # # # 

    lights_off_df = preds_df[lights_off_fltr].groupby(['subject_id', 'sleep_episode_id'])['epoch_ts'].min().\
        reset_index().rename({'epoch_ts': 'lights_off_time'}, axis=1)  # timestamp of the first lights_off_period epoch

    asleep_fltr = (preds_df[source_col_name] == 1)
    sleep_onset_df = preds_df[lights_off_fltr & asleep_fltr].groupby(['subject_id', 'sleep_episode_id'])['epoch_ts'].min().\
        reset_index().rename({'epoch_ts': 'sleep_onset'}, axis=1)  # timestamp of the first lights_off_period sleep epoch

    SOL_df = pd.merge(
        left=lights_off_df,
        right=sleep_onset_df,
        on=['subject_id', 'sleep_episode_id'],
        how='left'
    )
    SOL_df['SOL (Min)'] = (SOL_df['sleep_onset'] - SOL_df['lights_off_time']).dt.seconds / 60  # convert to minutes
    SOL_df = SOL_df.drop(['lights_off_time'], axis=1)  # keep sleep_onset for WASO

    # # # # # # # # # # # # # # # # # # # # 
    # #   WASO: Wake After Sleep Onset
    # #   Defined as the time in minutes of epochs scored as wake from SOL until lights on
    # # # # # # # # # # # # # # # # # # # # 

    # We need to find first wake after sleep onset. Let's first remove epochs before sleep onset
    first_wake_df = pd.merge(  # bringing in sleep onset (binary indicator) column
        left=preds_df[lights_off_fltr],
        right=SOL_df.drop('SOL (Min)', axis=1),  # don't need this column
        on=['subject_id', 'sleep_episode_id'],
        how='left'
    )
    # Sleep onset column is specific to each subject and sleep episode
    # So, we don't need to worry about grouping or filtering by these here
    # This is a row-wise comparison
    after_sleep_onset_df = first_wake_df[first_wake_df['epoch_ts'] > first_wake_df['sleep_onset']]  # remove epochs that occur before sleep onset
    awake_fltr = (after_sleep_onset_df[source_col_name] == 0)
    WASO_df = after_sleep_onset_df[awake_fltr].groupby(['subject_id', 'sleep_episode_id'])['epoch_ts'].count() / 2  # 2 epochs = 1 minutes
    WASO_df = WASO_df.reset_index().rename({'epoch_ts': 'WASO (Min)'}, axis=1)

    # bring in sleep onset timestamp to calculate time between sleep onset and first wake
    # WASO_df = pd.merge(
    #     left=first_wake_df,
    #     right=sleep_onset_df,
    #     on=['subject_id', 'sleep_episode_id'],
    #     how='left'
    # )

    # WASO_df['WASO (Min)'] = (WASO_df['first_wake'] - WASO_df['sleep_onset']).dt.seconds / 60  # convert to mintues
    # WASO_df = WASO_df.drop(['first_wake', 'sleep_onset'], axis=1)

    # Now that we're done with WASO we can drop sleep_onset from SOL
    SOL_df = SOL_df.drop('sleep_onset', axis=1)

    # # # # # # # # # # # # # # # # # # # # 
    # #   SEEF: Sleep Efficiency
    # #   Defined as the percentage of TST against TRT
    # # # # # # # # # # # # # # # # # # # # 

    SEEF_df = pd.merge(
        left=TRT_df,
        right=TST_df,
        on=['subject_id', 'sleep_episode_id'],
        how='left'
    )

    SEEF_df['SEEF'] = SEEF_df['TST (Min)'] / SEEF_df['TRT (Min)']
    SEEF_df = SEEF_df.drop(['TRT (Min)', 'TST (Min)'], axis=1)
    
    metrics_list = [TRT_df, TST_df, SEEF_df, SOL_df, WASO_df]

    merge_fn = lambda l, r: pd.merge(
        left=l,
        right=r,
        on=['subject_id', 'sleep_episode_id'],
        how='left'
    )

    sleep_summary_metrics = reduce(merge_fn, metrics_list)

    return sleep_summary_metrics

In [22]:
# Load participation dates to find the lab-day
# The lab-day is the last day for all subjects
# So we can find the time that lab day starts by subtracting 1 Day (exactly 24 hours)
# from their end timestamp

# We need to remove lab day from metric calculations (at least for now)
# Becuase we don't have proper predictions for lab day (used for training)
# there are rare cases where a few epochs were not in training and so they appear in the test set
# And not removing them could bias our metric values
# So, I explicitly remove the entire lab day before calculating sleep summary metrics

participation_dates = pd.read_csv('data/participation_dates.csv')

for dt_col in ['start_timestamp', 'end_timestamp']:
    participation_dates[dt_col] = pd.to_datetime(participation_dates[dt_col])

participation_dates['lab_day_start'] = pd.to_datetime(participation_dates['end_timestamp'])
participation_dates['lab_day_start'] = participation_dates['end_timestamp'] - np.timedelta64(1, 'D')
participation_dates = participation_dates[['subject_id', 'start_timestamp', 'lab_day_start']]

# Remove epochs after the timestamp that marks the start of the lab day
# See comment above
preds_df = pd.merge(
    left=all_preds,
    right=participation_dates,
    on='subject_id',
    how='left'
)
valid_data_fltr = (preds_df['epoch_ts'].between(preds_df['start_timestamp'], preds_df['lab_day_start'], inclusive='left'))
preds_df = preds_df[valid_data_fltr].drop('lab_day_start', axis=1)

# Now calculating the metrics
source_cols = ['pred_PSG-CNN', 'Biobank Sleep', 'AWS Sleep']
metrics_df = pd.DataFrame()
for source in source_cols:
    # source_metrics_df = calculate_sleep_summary_metrics(preds_df, source_col_name=source)
    source_metrics_df = calculate_sleep_summary_metrics_v2(preds_df, source_col_name=source)
    # for col in [c for c in source_metrics_df.columns if c not in ['subject_id', 'sleep_episode_id']]:
    #     source_metrics_df = source_metrics_df.rename({col: col + f'-{source}'}, axis=1)
    source_metrics_df.insert(0, 'source', source)
    metrics_df = pd.concat([metrics_df, source_metrics_df])

# metrics_df.to_excel('sleep_metrics.xlsx', index=False)

In [23]:
metrics_df.drop(['subject_id', 'sleep_episode_id'], axis=1).groupby('source').agg(['mean', 'std']).round(2)

TRT (Min)        TST (Min)          SEEF       SOL (Min)         \
                   mean    std      mean     std  mean   std      mean    std   
source                                                                          
AWS Sleep        448.79  86.78    374.24  103.04  0.84  0.18      3.73   7.05   
Biobank Sleep    448.79  86.78    391.89  126.93  0.87  0.23      7.20  13.89   
pred_PSG-CNN     448.79  86.78    289.79  106.10  0.65  0.21      8.02  16.99   

              WASO (Min)         
                    mean    std  
source                           
AWS Sleep          54.99  32.97  
Biobank Sleep      23.46  24.30  
pred_PSG-CNN      135.16  75.16

In [8]:
# # preds_df.groupby('sleep_episode_id')['Biobank Sleep'].transform(lambda x: x.isna().sum())
# temp = preds_df[preds_df['sleep_episode_id'] > 0].copy()
# temp['biobank_nan'] = temp['Biobank Sleep'].isna()
# temp['episode_has_nan'] = temp.groupby(['subject_id', 'sleep_episode_id'])['biobank_nan'].transform('sum').gt(0)

In [9]:
# temp = temp[~temp['episode_has_nan']]

# # Now calculating the metrics
# source_cols = ['pred_PSG-CNN', 'Biobank Sleep', 'AWS Sleep']
# metrics_df = pd.DataFrame()
# for source in source_cols:
#     source_metrics_df = calculate_sleep_summary_metrics(temp, source_col_name=source)
#     source_metrics_df.insert(0, 'source', source)
#     metrics_df = pd.concat([metrics_df, source_metrics_df])

# Bland-Altman plots

In [25]:
model_metrics_df = metrics_df[metrics_df['source'] == 'pred_PSG-CNN']
aws_metrics_df = metrics_df[metrics_df['source'] == 'AWS Sleep']
biobank_metrics_df = metrics_df[metrics_df['source'] == 'Biobank Sleep']

,source,subject_id,sleep_episode_id,TRT (Min),TST (Min),SEEF,SOL (Min),WASO (Min)
0,Biobank Sleep,1,1,420.0,357.0,0.850000,0.5,NaN
1,Biobank Sleep,1,2,355.0,355.0,1.000000,0.0,NaN
2,Biobank Sleep,1,3,370.0,359.0,0.970270,11.0,NaN
3,Biobank Sleep,1,4,450.0,380.0,0.844444,61.5,8.5
4,Biobank Sleep,1,5,430.0,362.5,0.843023,1.5,NaN
...,...,...,...,...,...,...,...,...
317,Biobank Sleep,36,8,470.0,403.5,0.858511,0.0,66.5
318,Biobank Sleep,36,9,428.0,364.5,0.851636,63.5,NaN
319,Biobank Sleep,36,10,527.0,458.5,0.870019,17.0,51.5
320,Biobank Sleep,36,11,512.0,484.0,0.945312,3.0,25.0
